In [78]:
import pandas as pd
from Bio import GenBank
from Bio import SeqIO

In [307]:
blast_hits_df = pd.read_csv("/Users/jakub/Desktop/internship_helmholtz/P3/blasthits_.csv", header = 0, delimiter = ";")

In [308]:
blast_hits_df

,Unnamed: 0,P13,P50071,P3,P4,P8,P3SEQ,P4SEQ,P8SEQ
0,NaN,5217773-5217842,5277080-5277149,36282-36640,35111-35180,12214-12283,NODE_35,NODE_1,NODE_15
1,NaN,1482792-1482843,5248784-5248813,22166-22235,71223-71274,76177-76228,NODE_2,NODE_24,NODE_21
2,NaN,5017910-5017938,5486508-5486536,18870-18916,277365-277393,286437-286465,NODE_65,NODE_2,NODE_1
3,NaN,4199733-4199780,2460746-2460797,69827-69878,26193-26220,3991-4018,NODE_4,NODE_28,NODE_67
4,NaN,X,6218014-6218061,70317-70373,141263-141310,141263-141310,NODE_20,NODE_9,NODE_8
5,NaN,NaN,X,61304-61332,63447-63476,40550-40579,NODE_3,NODE_1,NODE_15
6,NaN,NaN,NaN,147268-147315,X,X,NODE_6,NaN,NaN
7,NaN,NaN,NaN,34852-34882,NaN,NaN,NODE_35,NaN,NaN
8,NaN,NaN,NaN,50503-50532,NaN,NaN,NODE_2,NaN,NaN
9,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN


In [309]:
dict_of_hits = {}
dict_of_hits["P13"] = []
dict_of_hits["P50071"] = []
dict_of_hits["P3"] = []
dict_of_hits["P4"] = []
dict_of_hits["P8"] = []
dict_of_hits["P3SEQ"] = []
dict_of_hits["P4SEQ"] = []
dict_of_hits["P8SEQ"] = []

for (colname, colval) in blast_hits_df.iteritems():
    if colname in dict_of_hits.keys():
        dict_of_hits[colname].extend(colval.values)
    else:
        continue

In [310]:
print(dict_of_hits.keys()) 

dict_keys(['P13', 'P50071', 'P3', 'P4', 'P8', 'P3SEQ', 'P4SEQ', 'P8SEQ'])


In [311]:
print(dict_of_hits.values()) 

dict_values([['5217773-5217842', '1482792-1482843', '5017910-5017938', '4199733-4199780', 'X', nan, nan, nan, nan, nan], ['5277080-5277149', '5248784-5248813', '5486508-5486536', '2460746-2460797', '6218014-6218061', 'X', nan, nan, nan, nan], ['36282-36640', '22166-22235', '18870-18916', '69827-69878', '70317-70373', '61304-61332', '147268-147315', '34852-34882', '50503-50532', 'X'], ['35111-35180', '71223-71274', '277365-277393', '26193-26220', '141263-141310', '63447-63476', 'X', nan, nan, nan], ['12214-12283', '76177-76228', '286437-286465', '3991-4018', '141263-141310', '40550-40579', 'X', nan, nan, nan], ['NODE_35', 'NODE_2', 'NODE_65', 'NODE_4', 'NODE_20', 'NODE_3', 'NODE_6', 'NODE_35', 'NODE_2', nan], ['NODE_1', 'NODE_24', 'NODE_2', 'NODE_28', 'NODE_9', 'NODE_1', nan, nan, nan, nan], ['NODE_15', 'NODE_21', 'NODE_1', 'NODE_67', 'NODE_8', 'NODE_15', nan, nan, nan, nan]])


In [312]:
def range_subset(range1, range2):
    """Whether range1 is a subset of range2."""
    if not range1:
        return True  # empty range is subset of anything
    if not range2:
        return False  # non-empty range can't be subset of empty range
    if len(range1) > 1 and range1.step % range2.step:
        return False  # must have a single value or integer multiple step
    return range1.start in range2 and range1[-1] in range2

In [313]:
def check_the_hit(bacteria, gbkfile, hit_dict):
    allCDSs = []
    notfoundCDSs = []
    print("XXXXXXXXXXXXXX", bacteria, "XXXXXXXXXXXXXX")
    for hit in hit_dict[bacteria]:
        found_stat = False
        if hit == "X":
            break
        else:
            hit_start = hit.split("-")[0]
            hit_stop = hit.split("-")[1]
            print("The length of a hit in question: ", int(hit_stop) - int(hit_start))
            
            GenomeSeq = SeqIO.read(gbkfile, "genbank")

            for i in range(len(GenomeSeq.features)):
                current_feature = GenomeSeq.features[i]
                if current_feature.type == 'CDS':
                    cf_location = str(current_feature.location)
                    gbk_start_ = cf_location.split(":")[0]
                    gbk_start = int(gbk_start_.split("[")[-1])
                    gbk_stop_ = cf_location.split(":")[1]
                    gbk_stop = int(gbk_stop_.split("]")[0])
                    if (gbk_start, gbk_stop, current_feature.qualifiers['product']) not in allCDSs:
                        allCDSs.append((gbk_start, gbk_stop, current_feature.qualifiers['product']))
                    if range_subset(range(int(hit_start), int(hit_stop)), range(int(gbk_start), int(gbk_stop))) == True:
                        print("Hit found within a CDS")
                        found_stat = True
                        print(hit_start, hit_stop, "in", gbk_start, gbk_stop, current_feature.qualifiers['product'])
                        
            
        if found_stat == False:
            notfoundCDSs.append((int(hit_start), int(hit_stop)))
    
    # Now for the ones outside of CDSs    
    
    CDSsorted_up = sorted(allCDSs)
    for i in range(len(notfoundCDSs)):
        possibilities_up = []
        possibilities_down = []
        for j in range(len(CDSsorted_up)):
            #print(CDSsorted_up[j])
            if notfoundCDSs[i][0] > CDSsorted_up[j][0]:
                pass
            else:
                possibilities_up.extend([CDSsorted_up[j], CDSsorted_up[j+1], CDSsorted_up[j+2], CDSsorted_up[j+3], CDSsorted_up[j+5]])
                possibilities_down.extend([CDSsorted_up[j-1], CDSsorted_up[j-2], CDSsorted_up[j-3], CDSsorted_up[j-4], CDSsorted_up[j-5]])
                break

        print("For this hit ",  notfoundCDSs[i], "found upstream: ", possibilities_up)
        print("For this hit ",  notfoundCDSs[i], "found downstream: ", possibilities_down)
    

In [314]:
check_the_hit("P13", "/Users/jakub/Desktop/internship_helmholtz/P3/final_data2/raw/genbank/p13.gbk", dict_of_hits)

XXXXXXXXXXXXXX P13 XXXXXXXXXXXXXX
The length of a hit in question:  69
The length of a hit in question:  51
Hit found within a CDS
1482792 1482843 in 1479287 1483241 ['RHS repeat protein']
The length of a hit in question:  28
Hit found within a CDS
5017910 5017938 in 5017746 5019720 ['methyl-accepting chemotaxis protein']
The length of a hit in question:  47
Hit found within a CDS
4199733 4199780 in 4199071 4200361 ['MFS transporter']
For this hit  (5217773, 5217842) found upstream:  [(5217883, 5218732, ["4-(cytidine 5'-diphospho)-2-C-methyl-D-erythritol kinase"]), (5218733, 5219351, ['lipoprotein localization protein LolB']), (5219355, 5221128, ['tetratricopeptide repeat protein']), (5221271, 5222540, ['glutamyl-tRNA reductase']), (5223641, 5224472, ['peptide chain release factor N(5)-glutamine methyltransferase'])]
For this hit  (5217773, 5217842) found downstream:  [(5216775, 5217717, ['ribose-phosphate pyrophosphokinase']), (5216044, 5216659, ['50S ribosomal protein L25']), (521541

In [315]:
check_the_hit("P50071", "/Users/jakub/Desktop/internship_helmholtz/P3/final_data2/raw/genbank/p50071.gbk", dict_of_hits)

XXXXXXXXXXXXXX P50071 XXXXXXXXXXXXXX
The length of a hit in question:  69
The length of a hit in question:  29
The length of a hit in question:  28
Hit found within a CDS
5486508 5486536 in 5484725 5486699 ['chemotaxis protein']
The length of a hit in question:  51
Hit found within a CDS
2460746 2460797 in 2460347 2464301 ['Rhs family protein']
The length of a hit in question:  47
Hit found within a CDS
6218014 6218061 in 6217432 6218722 ['citrate-proton symporter']
For this hit  (5277080, 5277149) found upstream:  [(5277204, 5278146, ['ribose-phosphate pyrophosphokinase']), (5278262, 5278877, ['50S ribosomal protein L25']), (5278918, 5279503, ['peptidyl-tRNA hydrolase']), (5279543, 5280644, ['GTP-binding protein']), (5281972, 5284201, ['TonB-dependent receptor'])]
For this hit  (5277080, 5277149) found downstream:  [(5276189, 5277038, ['kinase']), (5275570, 5276188, ['membrane protein']), (5273838, 5275566, ['hypothetical protein']), (5272381, 5273650, ['glutamyl-tRNA reductase']), (5

In [338]:
def check_the_hit_assembly(bacteria, node_col_id, gbkfile, hit_dict):
    allCDSs = []
    notfoundCDSs = []
    print("XXXXXXXXXXXXXX", bacteria, "XXXXXXXXXXXXXX")
    for h in range(len(hit_dict[bacteria])):
        found_stat = False
        if hit_dict[bacteria][h] == "X":
            break
        else:
            hit_start = hit_dict[bacteria][h].split("-")[0]
            hit_stop = hit_dict[bacteria][h].split("-")[1]
            print("The length of a hit in question: ", int(hit_stop) - int(hit_start)) 
            for record in SeqIO.parse(gbkfile, "genbank"):
                for i in range(len(record.features)):
                    current_feature = record.features[i]
                    if current_feature.type == 'CDS':
                        cf_location = str(current_feature.location)
                        gbk_start_ = cf_location.split(":")[0]
                        if "<" in gbk_start_:
                            gbk_start_ = gbk_start_.split('<')[-1]
                        gbk_start = int(gbk_start_.split("[")[-1])
                        gbk_stop_ = cf_location.split(":")[1]
                        if ">" in gbk_stop_:
                            gbk_stop_ = gbk_stop_.split('>')[-1]
                        gbk_stop = int(gbk_stop_.split("]")[0])
                        if (gbk_start, gbk_stop, current_feature.qualifiers['product'], record.id) not in allCDSs:
                            allCDSs.append((gbk_start, gbk_stop, current_feature.qualifiers['product'], record.id))
                        if range_subset(range(int(hit_start), int(hit_stop)), range(int(gbk_start), int(gbk_stop))) == True:
                            if record.id == hit_dict[node_col_id][h]:                                
                                print("Hit found within a CDS")
                                found_stat = True
                                print(hit_start, hit_stop, "in", gbk_start, gbk_stop, current_feature.qualifiers['product'])
                        
            
        if found_stat == False:
            notfoundCDSs.append((int(hit_start), int(hit_stop), hit_dict[node_col_id][h]))
    
    #Now for the ones outside of CDSs     
    
    CDSsorted_up = sorted(allCDSs)
    for i in range(len(notfoundCDSs)):
        possibilities_up = []
        possibilities_down = []
        for j in range(len(CDSsorted_up)):
            if notfoundCDSs[i][-1] == CDSsorted_up[j][-1]:
                if notfoundCDSs[i][0] > CDSsorted_up[j][0]:
                    pass
                else:
                    possibilities_up.extend([CDSsorted_up[j], CDSsorted_up[j+1], CDSsorted_up[j+2], CDSsorted_up[j+3], CDSsorted_up[j+5]])
                    possibilities_down.extend([CDSsorted_up[j-1], CDSsorted_up[j-2], CDSsorted_up[j-3], CDSsorted_up[j-4], CDSsorted_up[j-5]])
                    break

        print("For this hit ",  notfoundCDSs[i], "found upstream: ", possibilities_up)
        print("For this hit ",  notfoundCDSs[i], "found downstream: ", possibilities_down)

In [339]:
check_the_hit_assembly("P3", "P3SEQ", "/Users/jakub/Desktop/internship_helmholtz/P3/final_data2/raw/genbank/p3.gbk", dict_of_hits)

XXXXXXXXXXXXXX P3 XXXXXXXXXXXXXX
The length of a hit in question:  358
Hit found within a CDS
36282 36640 in 35711 36683 ['Major capsid protein 13-like']
The length of a hit in question:  69
The length of a hit in question:  46
Hit found within a CDS
18870 18916 in 18808 22654 ['cobaltochelatase subunit CobN']
The length of a hit in question:  51
Hit found within a CDS
69827 69878 in 66322 70276 ['Toxin protein Tse5']
The length of a hit in question:  56
The length of a hit in question:  28
Hit found within a CDS
61304 61332 in 59521 61495 ['Methyl-accepting chemotaxis protein CtpL']
The length of a hit in question:  47
Hit found within a CDS
147268 147315 in 146606 147896 ['citrate-proton symporter']
The length of a hit in question:  30
The length of a hit in question:  29
For this hit  (22166, 22235, 'NODE_2') found upstream:  [(22276, 23125, ["4-(cytidine 5'-diphospho)-2-C-methyl-D-erythritol kinase"], 'NODE_2'), (22285, 22774, ['DUF1833 domain-containing protein'], 'NODE_53'), (222

In [340]:
check_the_hit_assembly("P4", "P4SEQ","/Users/jakub/Desktop/internship_helmholtz/P3/final_data2/raw/genbank/p4.gbk", dict_of_hits)

XXXXXXXXXXXXXX P4 XXXXXXXXXXXXXX
The length of a hit in question:  69
The length of a hit in question:  51
Hit found within a CDS
71223 71274 in 67718 71672 ['Toxin protein Tse5']
The length of a hit in question:  28
Hit found within a CDS
277365 277393 in 277201 279175 ['Methyl-accepting chemotaxis protein CtpL']
The length of a hit in question:  27
Hit found within a CDS
26193 26220 in 26062 27730 ['DNA helicase']
The length of a hit in question:  47
Hit found within a CDS
141263 141310 in 140601 141891 ['citrate-proton symporter']
The length of a hit in question:  29
For this hit  (35111, 35180, 'NODE_1') found upstream:  [(35221, 36070, ["4-(cytidine 5'-diphospho)-2-C-methyl-D-erythritol kinase"], 'NODE_1'), (35224, 35758, ['translation initiation factor IF-3'], 'NODE_28'), (35260, 36121, ['carboxylate/amino acid/amine transporter'], 'NODE_45'), (35275, 36724, ['two-component system sensor histidine kinase GtrS'], 'NODE_50'), (35327, 35531, ['YscE family type III secretion system c

In [341]:
check_the_hit_assembly("P8", "P8SEQ","/Users/jakub/Desktop/internship_helmholtz/P3/final_data2/raw/genbank/p8.gbk", dict_of_hits)

XXXXXXXXXXXXXX P8 XXXXXXXXXXXXXX
The length of a hit in question:  69
The length of a hit in question:  51
Hit found within a CDS
76177 76228 in 72672 76626 ['Toxin protein Tse5']
The length of a hit in question:  28
Hit found within a CDS
286437 286465 in 286273 288247 ['Methyl-accepting chemotaxis protein CtpL']
The length of a hit in question:  27
Hit found within a CDS
3991 4018 in 2480 4148 ['DNA helicase']
The length of a hit in question:  47
Hit found within a CDS
141263 141310 in 140601 141891 ['citrate-proton symporter']
The length of a hit in question:  29
For this hit  (12214, 12283, 'NODE_15') found upstream:  [(12324, 13173, ["4-(cytidine 5'-diphospho)-2-C-methyl-D-erythritol kinase"], 'NODE_15'), (12329, 12950, ['DUF2170 domain-containing protein'], 'NODE_34'), (12344, 13046, ['nuclease'], 'NODE_50'), (12350, 13478, ['FAD-dependent catabolic D-arginine dehydrogenase DauA'], 'NODE_53'), (12405, 12912, ['peptidoglycan-associated lipoprotein Pal'], 'NODE_61')]
For this hit  